In [12]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# !python3 -m spacy download en
from spacy.lang.es import Spanish
# !python3 -m spacy download es_core_news_md


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_sp=set(stopwords.words('spanish'))
stop_words_en=set(stopwords.words('english'))
stop_words=stop_words_sp | stop_words_en


import re

from langdetect import detect

from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhatch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
url_base = "https://www.gob.mx/presidencia/es/archivo/articulos?idiom=es&order=DESC&page="

In [14]:
def obtiene_links(page):
    response = requests.get(url_base + str(page))
    sopa = BeautifulSoup(response.text)

    div_prensa = sopa.find(id='prensa')
    anchors = []
    if div_prensa is not None:
        anchors = div_prensa.find_all("a")
    else:
        # Esto hace que el scraping funcione aunque la página regrese JavaScript
        lineas = [linea.strip() for linea in response.text.split("\n") if "prensa" in linea]
        lineas_limpias = [linea[21:-2].replace('\\\"',"\"").replace('\/',"/").replace('\\n',"") for linea in lineas]
        for html_bonito in lineas_limpias:
            sopita = BeautifulSoup(html_bonito)
            anclas = sopita.find_all("a")
            anchors.extend(anclas)

    hrefs = ["https://www.gob.mx" + a["href"] for a in anchors]
    return hrefs

In [15]:
links = []
for pg in range(106):
    links.extend(obtiene_links(pg))

In [16]:
pres = []
for link in links:
    if link.

['https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-informe-diario-sobre-coronavirus-covid-19-en-mexico-252065?idiom=es',
 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-7-de-septiembre-de-2020?idiom=es',
 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-informe-diario-sobre-coronavirus-covid-19-en-mexico-251971?idiom=es',
 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-informe-diario-sobre-coronavirus-covid-19-en-mexico-251937?idiom=es',
 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-informe-diario-sobre-coronavirus-covid-19-en-mexico-251902?idiom=es',
 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-4-de-septiembre-de-2020?idiom=es',
 'https://www.go